In [ ]:
# ! pip install fastparquet

In [2]:
import pandas as pd
import swifter
import numpy as np
from glob import glob
import json
import gc


In [3]:
train = pd.read_parquet("../data_phase1/train.parquet")
val = pd.read_parquet("../data_phase1/validation.parquet")


In [4]:
attribute_df = pd.read_parquet("../data_phase1/attributes.parquet")

In [5]:
train = train.merge(attribute_df, on='product_id', how='left')
val = val.merge(attribute_df, on='product_id', how='left')


In [6]:
train.head(2)

,query_id,user_id,session_id,product_id,page_type,previous_page_type,device_category,device_platform,user_tier,user_country,...,season,collection,category_id_l1,category_id_l2,category_id_l3,brand_id,season_year,start_online_date,material_values,attribute_values
0,92d4dd491a874a2cf92c8d311a44a42b597c64a5ede23d...,e5e4c71b1b9456dafece1338762d4ee3db698cf32c384c...,2740b0d77b4e6fafd75321f7d0794210afa8bd650955e7...,bf056e3841dd3a358c6aacb1f9e74e4c7c4adc62e33b45...,596618814963e496d74434df8b8fe3306892f2e4ce6aaa...,c9f34437ce0e536fefd11a34b9a411b541d2dabfec872a...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,fec032cb05435471f2305006f4a1ba994c9d2f4bcad8ef...,...,9db35d237f5873f0410d3ca18c07430270086eb1e7838d...,4c8006c7e513057a9138641abc2a9e65f4b014a8045259...,642878c18e40a1028b6610cd3396d4ee4fb6ad9623f5bb...,a37fdccd28723fba01a2f0a68d3bb1a3b847e550a7b0c2...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...,206b88b91a241cc6ed5af856ce3ddda4dccf91ae2ec798...,dc1194ba428d5cd4c49f8a769a6577ac1042162da38bc1...,570.041667,f61ecea9b45f1590e57706b88207449bdd4cb703b917ad...,28d2b1e7b0970e2c58966c47b4c7ab7426e43a8689007e...
1,541a93bd95c3f4127a53e6b0d4b41db55ad9cb9e19d34a...,fca847f7eb5a5a21991421354b0f26afb4a517e540541f...,d5feab37634fd140e85b8f98dcb909a8779b4f0417c73a...,c6513ec49c8e04c265c907933799ff76f24c075c6308c4...,06a7f8e972f61aeb0e06335699518079a444e4450ff766...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d179859aac8f7c1f88e1ee29b6655596873318c55127d3...,8c82855f15d05cd74fa59956434df17522fc68e4ce3900...,...,847a067597e39838f1f85b0774f44e68b4d6e64d3ec4dd...,4c8006c7e513057a9138641abc2a9e65f4b014a8045259...,f6bafdefb3c04b392adc1d139ea55a565c8829627bb7d3...,884c59e2814e352d318a8447c2f32a0370955af64838e3...,393b930ad3c40c6af1c14ce8572f39a3a2027ef70dd21e...,de60eee3c057eab83a9987275b3ecfb080026c3297b036...,ef7d8d2e02aec8c328dafde95486f7181c37d07be3d167...,208.041667,c4aebf95aa0044c5230c104faa9ef2b5ccfab836b764d4...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...


In [7]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df


In [8]:
train = reduce_mem_usage(train)
val = reduce_mem_usage(val)


Memory usage of dataframe is 802.92 MB
Memory usage after optimization is: 274.46 MB
Decreased by 65.8%
Memory usage of dataframe is 152.04 MB
Memory usage after optimization is: 67.06 MB
Decreased by 55.9%


In [9]:
gc.collect()

22

In [10]:
train.shape, val.shape

((3507990, 29), (687192, 28))

In [11]:
BASE_PATH = "../preprocessed_data/"

In [12]:
merge_paths = list(set(glob(BASE_PATH+"*.csv")) - \
                   set([BASE_PATH+"session_wise_product_attribute_frequency_agg_features.csv", \
                        BASE_PATH+"product_click_features_by_session.csv", \
                        BASE_PATH+"product_material_attribute_one-hot.csv"]))



In [13]:
merge_paths

['../preprocessed_data/brand_id_freq_data.csv',
 '../preprocessed_data/collection_popularity_df.csv',
 '../preprocessed_data/gender_frequency.csv',
 '../preprocessed_data/collection_freq_data.csv',
 '../preprocessed_data/main_colour_popularity_df.csv',
 '../preprocessed_data/unique_user_interactions_per_product.csv',
 '../preprocessed_data/query_wise_price_features.csv',
 '../preprocessed_data/season_year_freq_data.csv',
 '../preprocessed_data/season_frequency.csv',
 '../preprocessed_data/category_id_l1_popularity_df.csv',
 '../preprocessed_data/second_colour_popularity_df.csv',
 '../preprocessed_data/global_product_frequency.csv',
 '../preprocessed_data/query_wise_start_online_date_features.csv',
 '../preprocessed_data/user_time_features.csv',
 '../preprocessed_data/category_id_l2_popularity_df.csv',
 '../preprocessed_data/brand_popularity_df.csv',
 '../preprocessed_data/second_colour_frequency.csv',
 '../preprocessed_data/category_id_l1_freq_data.csv',
 '../preprocessed_data/user_tie

In [14]:
len(merge_paths)

21

In [15]:
done_files = []

In [16]:
def merge_data(train_data, val_data, merge_data_path_list, merge_cols='', flag=0):
    compulsory_cols = ["query_id", "user_id", "session_id", "product_id"]
    for path in merge_data_path_list:
        print("Merging file...", path.split('/')[-1])
        prev_cols_train = set(train_data.columns.tolist())
        prev_cols_val = set(val_data.columns.tolist())
        d = pd.read_csv(path)
#         d = reduce_mem_usage(d)
        if 'is_click' in d.columns.tolist():
            d = d.drop('is_click', 1)
        if flag==0:
            merge_cols = [d.columns[0]]
            
        for col in d.columns.tolist():
            if col in train.columns.tolist() and col not in compulsory_cols and col not in merge_cols:
                d = d.drop(col, 1)
            
        train_data = train_data.merge(d, on=merge_cols, how='left')
        val_data = val_data.merge(d, on=merge_cols, how='left')
        print("Train: ", train_data.shape, "Val: ", val_data.shape)
        done_files.append(path)
        print("Added columns train: ", set(train_data.columns.tolist()) - prev_cols_train)
        print("Added columns val: ", set(val_data.columns.tolist()) - prev_cols_val)
        del d
        gc.collect()
    
    return train_data, val_data


In [17]:
# d = pd.read_csv("../preprocessed_data/brand_id_freq_data.csv")
# d.columns[0]
# d.head(2)

In [18]:
train, val = merge_data(train, val, merge_paths[:6])


Merging file... brand_id_freq_data.csv
Train:  (3507990, 30) Val:  (687192, 29)
Added columns train:  {'brand_id_perc'}
Added columns val:  {'brand_id_perc'}
Merging file... collection_popularity_df.csv
Train:  (3507990, 33) Val:  (687192, 32)
Added columns train:  {'collection_click_count', 'collection_popularity_score', 'collection_impression_count'}
Added columns val:  {'collection_click_count', 'collection_popularity_score', 'collection_impression_count'}
Merging file... gender_frequency.csv
Train:  (3507990, 34) Val:  (687192, 33)
Added columns train:  {'gender_perc'}
Added columns val:  {'gender_perc'}
Merging file... collection_freq_data.csv
Train:  (3507990, 35) Val:  (687192, 34)
Added columns train:  {'collection_perc'}
Added columns val:  {'collection_perc'}
Merging file... main_colour_popularity_df.csv
Train:  (3507990, 38) Val:  (687192, 37)
Added columns train:  {'main_colour_impression_count', 'main_colour_click_count', 'main_colour_popularity_score'}
Added columns val: 

In [19]:
train.columns

Index(['query_id', 'user_id', 'session_id', 'product_id', 'page_type',
       'previous_page_type', 'device_category', 'device_platform', 'user_tier',
       'user_country', 'context_type', 'context_value', 'product_price',
       'week', 'week_day', 'is_click', 'gender', 'main_colour',
       'second_colour', 'season', 'collection', 'category_id_l1',
       'category_id_l2', 'category_id_l3', 'brand_id', 'season_year',
       'start_online_date', 'material_values', 'attribute_values',
       'brand_id_perc', 'collection_impression_count',
       'collection_click_count', 'collection_popularity_score', 'gender_perc',
       'collection_perc', 'main_colour_impression_count',
       'main_colour_click_count', 'main_colour_popularity_score',
       '#unique_users_interacted'],
      dtype='object')

In [20]:
[each.split('/')[-1] for each in done_files]


['brand_id_freq_data.csv',
 'collection_popularity_df.csv',
 'gender_frequency.csv',
 'collection_freq_data.csv',
 'main_colour_popularity_df.csv',
 'unique_user_interactions_per_product.csv']

In [21]:
# done_files = [BASE_PATH+each for each in done_files]
# done_files

In [22]:
train.shape, val.shape

((3507990, 39), (687192, 38))

In [23]:
train.memory_usage().sum()/(1024*1024*1024), val.memory_usage().sum()/(1024*1024*1024)

(0.6117381323128939, 0.12694331631064415)

In [24]:
# train.to_csv("../preprocessed_data/merged_data/train_static_merged_v1.csv", index=False, chunksize=10000)

In [25]:
# val.to_csv("../preprocessed_data/merged_data/val_static_merged_v1.csv", index=False)

In [26]:
merge_paths = list(set(merge_paths) - set(done_files))
merge_paths

['../preprocessed_data/second_colour_popularity_df.csv',
 '../preprocessed_data/global_product_frequency.csv',
 '../preprocessed_data/query_wise_start_online_date_features.csv',
 '../preprocessed_data/main_colour_frequency.csv',
 '../preprocessed_data/user_time_features.csv',
 '../preprocessed_data/category_id_l2_popularity_df.csv',
 '../preprocessed_data/brand_popularity_df.csv',
 '../preprocessed_data/second_colour_frequency.csv',
 '../preprocessed_data/query_wise_price_features.csv',
 '../preprocessed_data/category_id_l1_freq_data.csv',
 '../preprocessed_data/season_year_freq_data.csv',
 '../preprocessed_data/user_tier_features.csv',
 '../preprocessed_data/category_id_l2_freq_data.csv',
 '../preprocessed_data/season_frequency.csv',
 '../preprocessed_data/category_id_l1_popularity_df.csv']

In [27]:
train, val = merge_data(train, val, merge_paths[:7])


Merging file... second_colour_popularity_df.csv
Train:  (3507990, 42) Val:  (687192, 41)
Added columns train:  {'second_colour_click_count', 'second_colour_popularity_score', 'second_colour_impression_count'}
Added columns val:  {'second_colour_click_count', 'second_colour_popularity_score', 'second_colour_impression_count'}
Merging file... global_product_frequency.csv
Train:  (3507990, 45) Val:  (687192, 44)
Added columns train:  {'global_product_percentage', '#impressions_for_product', 'global_product_frequency'}
Added columns val:  {'global_product_percentage', '#impressions_for_product', 'global_product_frequency'}
Merging file... query_wise_start_online_date_features.csv
Train:  (3507990, 48) Val:  (687192, 47)
Added columns train:  {'max_query_start_online_date', 'min_query_start_online_date', 'mean_query_start_online_date'}
Added columns val:  {'max_query_start_online_date', 'min_query_start_online_date', 'mean_query_start_online_date'}
Merging file... main_colour_frequency.csv


In [28]:
merge_paths = list(set(merge_paths) - set(done_files))
merge_paths

['../preprocessed_data/second_colour_frequency.csv',
 '../preprocessed_data/query_wise_price_features.csv',
 '../preprocessed_data/category_id_l1_freq_data.csv',
 '../preprocessed_data/season_year_freq_data.csv',
 '../preprocessed_data/user_tier_features.csv',
 '../preprocessed_data/category_id_l2_freq_data.csv',
 '../preprocessed_data/season_frequency.csv',
 '../preprocessed_data/category_id_l1_popularity_df.csv']

In [29]:
train, val = merge_data(train, val, merge_paths)


Merging file... second_colour_frequency.csv
Train:  (3507990, 58) Val:  (687192, 57)
Added columns train:  {'second_colour_perc'}
Added columns val:  {'second_colour_perc'}
Merging file... query_wise_price_features.csv
Train:  (3507990, 61) Val:  (687192, 60)
Added columns train:  {'mean_query_price', 'max_query_price', 'min_query_price'}
Added columns val:  {'mean_query_price', 'max_query_price', 'min_query_price'}
Merging file... category_id_l1_freq_data.csv
Train:  (3507990, 62) Val:  (687192, 61)
Added columns train:  {'category_id_l1_perc'}
Added columns val:  {'category_id_l1_perc'}
Merging file... season_year_freq_data.csv
Train:  (3507990, 63) Val:  (687192, 62)
Added columns train:  {'season_year_perc'}
Added columns val:  {'season_year_perc'}
Merging file... user_tier_features.csv
Train:  (3507990, 75) Val:  (687192, 74)
Added columns train:  {'user_tier_min_click_product_start_online_date', 'user_tier_mean_product_start_online_date', 'user_tier_mean_click_product_price', 'us

In [30]:
train, val = merge_data(train, val, [BASE_PATH+"session_wise_product_attribute_frequency_agg_features.csv"], merge_cols=["session_id", "product_id"], flag=1)



Merging file... session_wise_product_attribute_frequency_agg_features.csv
Train:  (3507990, 93) Val:  (687192, 92)
Added columns train:  {'mean_session_start_online_date', 'session_category_id_l2_frequency', 'main_colour_freq_in session', 'gender_freq_in session', 'session_start_online_date_mean', 'second_colour_freq_in session', 'session_category_id_l1_frequency', 'session_brand_id_frequency', 'session_start_online_date_max', 'session_collection_frequency', 'session_season_frequency', 'count_session_items', 'session_start_online_date_min'}
Added columns val:  {'mean_session_start_online_date', 'session_category_id_l2_frequency', 'main_colour_freq_in session', 'gender_freq_in session', 'session_start_online_date_mean', 'second_colour_freq_in session', 'session_category_id_l1_frequency', 'session_brand_id_frequency', 'session_start_online_date_max', 'session_collection_frequency', 'session_season_frequency', 'count_session_items', 'session_start_online_date_min'}


In [31]:
train, val = merge_data(train, val, [BASE_PATH+"product_click_features_by_session.csv"], merge_cols=["session_id", "product_id"], flag=1)


Merging file... product_click_features_by_session.csv
Train:  (3507990, 99) Val:  (687192, 98)
Added columns train:  {'session_impression_count', 'mean_session_product_price', 'product_session_frequency', 'min_session_product_price', 'max_session_product_price', 'product_session_click_proportion'}
Added columns val:  {'session_impression_count', 'mean_session_product_price', 'product_session_frequency', 'min_session_product_price', 'max_session_product_price', 'product_session_click_proportion'}


In [32]:
train.columns

Index(['query_id', 'user_id', 'session_id', 'product_id', 'page_type',
       'previous_page_type', 'device_category', 'device_platform', 'user_tier',
       'user_country', 'context_type', 'context_value', 'product_price',
       'week', 'week_day', 'is_click', 'gender', 'main_colour',
       'second_colour', 'season', 'collection', 'category_id_l1',
       'category_id_l2', 'category_id_l3', 'brand_id', 'season_year',
       'start_online_date', 'material_values', 'attribute_values',
       'brand_id_perc', 'collection_impression_count',
       'collection_click_count', 'collection_popularity_score', 'gender_perc',
       'collection_perc', 'main_colour_impression_count',
       'main_colour_click_count', 'main_colour_popularity_score',
       '#unique_users_interacted', 'second_colour_impression_count',
       'second_colour_click_count', 'second_colour_popularity_score',
       'global_product_frequency', 'global_product_percentage',
       '#impressions_for_product', 'mean_query_s

In [33]:
train["diff_prod_price_from_user_tier_mean"] = train["product_price"] - train["user_tier_mean_product_price"]
train["diff_prod_start_online_date_from_user_tier_mean"] = train["product_price"] - train["user_tier_mean_product_start_online_date"]

val["diff_prod_price_from_user_tier_mean"] = val["product_price"] - val["user_tier_mean_product_price"]
val["diff_prod_start_online_date_from_user_tier_mean"] = val["product_price"] - val["user_tier_mean_product_start_online_date"]


In [34]:
train["diff_start_online_date_from_user_tier_clicked_mean"] = train["start_online_date"] - train["user_tier_mean_click_product_price"]
train["diff_start_online_date_from_user_tier_clicked_mean"] = train["start_online_date"] - train["user_tier_mean_click_product_start_online_date"]

val["diff_start_online_date_from_user_tier_clicked_mean"] = val["start_online_date"] - val["user_tier_mean_click_product_price"]
val["diff_start_online_date_from_user_tier_clicked_mean"] = val["start_online_date"] - val["user_tier_mean_click_product_start_online_date"]


In [35]:
train["diff_prod_price_from_session_mean"] = train["product_price"] - train["mean_session_product_price"]
train["diff_prod_price_from_query_mean"] = train["product_price"] - train["max_query_price"]

val["diff_prod_price_from_session_mean"] = val["product_price"] - val["mean_session_product_price"]
val["diff_prod_price_from_query_mean"] = val["product_price"] - val["max_query_price"]


In [36]:
dic = json.load(open("../preprocessed_data/average_click_values.json", "r"))


In [37]:
dic

{'mean_click_price': 0.0017881393432617188,
 'mean_click_start_online_date': 414.747518851918}

In [38]:
train["diff_prod_price_from_click_mean"] = train["product_price"] - dic["mean_click_price"]
val["diff_prod_price_from_click_mean"] = val["product_price"] - dic["mean_click_price"]


In [39]:
train["diff_start_online_date_from_session_mean"] = train["start_online_date"] - train["session_start_online_date_mean"]
train["diff_start_online_date_from_query_mean"] = train["start_online_date"] - train["mean_query_start_online_date"]

val["diff_start_online_date_from_session_mean"] = val["start_online_date"] - val["session_start_online_date_mean"]
val["diff_start_online_date_from_query_mean"] = val["start_online_date"] - val["mean_query_start_online_date"]


In [40]:
train["diff_start_online_date_from_click_mean"] = train["start_online_date"] - dic["mean_click_start_online_date"]
val["diff_start_online_date_from_click_mean"] = val["start_online_date"] - dic["mean_click_start_online_date"]


In [41]:
del attribute_df
gc.collect()


66

In [42]:
train_rank_features = pd.read_csv("../preprocessed_data/rank_features/train_rank_features_and_query_prod_list.csv")


In [43]:
val_rank_features = pd.read_csv("../preprocessed_data/rank_features/val_rank_features_and_query_prod_list.csv")


In [44]:
train_rank_features = train_rank_features[['query_id', 'user_id', 'session_id', 'product_id', 'price_rank', \
                                           'start_online_date_rank', 'user_step']]
val_rank_features = val_rank_features[['query_id', 'user_id', 'session_id', 'product_id', 'price_rank', \
                                           'start_online_date_rank', 'user_step']]


In [45]:
train = train.merge(train_rank_features, on=["query_id", "user_id", "session_id", "product_id"], how="left")
val = val.merge(val_rank_features, on=["query_id", "user_id", "session_id", "product_id"], how="left")


In [46]:
train_rank_features.columns

Index(['query_id', 'user_id', 'session_id', 'product_id', 'price_rank',
       'start_online_date_rank', 'user_step'],
      dtype='object')

In [47]:
train.shape, val.shape

((3507990, 111), (687192, 110))

In [48]:
# train = train.drop(["is_click_y"], 1)
# train = train.rename(columns = {"is_click_x": "is_click"})

In [49]:
set(train.columns)-set(val.columns)

{'is_click'}

In [50]:
is_click = train.is_click.values
train = train.drop(["is_click"], 1)
train["is_click"] = is_click


In [51]:
train.memory_usage().sum()/(1024*1024*1024), val.memory_usage().sum()/(1024*1024*1024)


(2.587738648056984, 0.5074375830590725)

In [52]:
# train.to_csv("../preprocessed_data/merged_data/train.csv", index=False, chunksize=10000)


In [53]:
train.head()

,query_id,user_id,session_id,product_id,page_type,previous_page_type,device_category,device_platform,user_tier,user_country,...,diff_prod_price_from_session_mean,diff_prod_price_from_query_mean,diff_prod_price_from_click_mean,diff_start_online_date_from_session_mean,diff_start_online_date_from_query_mean,diff_start_online_date_from_click_mean,price_rank,start_online_date_rank,user_step,is_click
0,92d4dd491a874a2cf92c8d311a44a42b597c64a5ede23d...,e5e4c71b1b9456dafece1338762d4ee3db698cf32c384c...,2740b0d77b4e6fafd75321f7d0794210afa8bd650955e7...,bf056e3841dd3a358c6aacb1f9e74e4c7c4adc62e33b45...,596618814963e496d74434df8b8fe3306892f2e4ce6aaa...,c9f34437ce0e536fefd11a34b9a411b541d2dabfec872a...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,fec032cb05435471f2305006f4a1ba994c9d2f4bcad8ef...,...,-0.002363,-1.989655e-02,-0.001525,73.171875,222.465278,155.25,3,5,2,0
1,541a93bd95c3f4127a53e6b0d4b41db55ad9cb9e19d34a...,fca847f7eb5a5a21991421354b0f26afb4a517e540541f...,d5feab37634fd140e85b8f98dcb909a8779b4f0417c73a...,c6513ec49c8e04c265c907933799ff76f24c075c6308c4...,06a7f8e972f61aeb0e06335699518079a444e4450ff766...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d179859aac8f7c1f88e1ee29b6655596873318c55127d3...,8c82855f15d05cd74fa59956434df17522fc68e4ce3900...,...,-0.025978,-1.105777e-01,0.011177,-33.944444,-56.034722,-206.75,1,1,19,0
2,263ea1e38126fe0c7bfbff24a33b1a09f4dac4f8cd4bb4...,90ea15d8d96a9d3e7ab463d990e5f4565cac9477498d37...,18ae37a1a05faa7fd54818794a1a8e44073e00a56fa05f...,b4d5e28da10318aa7776b364528dc92f83ba45326018b5...,06a7f8e972f61aeb0e06335699518079a444e4450ff766...,c9f34437ce0e536fefd11a34b9a411b541d2dabfec872a...,bf2241c08d92d32a6782b4041a2c11ca58882ca88454b3...,702e4598004745673c0f6b50387bef9e1d5f503bd8c1c0...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,c94ddcb9053eae77ab9abec807ef2b0bb4efac14404d17...,...,-0.000527,-1.756769e-03,0.000945,-211.541667,-208.375000,134.25,2,1,1,0
3,3727580d84ce2fbe42ff8bc6f732331f65ea659864a04c...,8f88d89f2a71e2adf42f885fa6adedd09bf039843b535c...,10667a5a6047aa173d13997cdcd996cbdaf9b0149f9655...,5a36f600d3c01763c28e2dafc53119fba7bcc6a867ab8f...,06a7f8e972f61aeb0e06335699518079a444e4450ff766...,c9f34437ce0e536fefd11a34b9a411b541d2dabfec872a...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,8c82855f15d05cd74fa59956434df17522fc68e4ce3900...,...,-0.000231,-1.554665e-04,-0.001718,322.381944,324.125000,166.25,1,6,1,0
4,1fcf5d263785455311cecf2f864eaa2eeca4da488383d9...,6b21688d90dfd9677fa7979dfc6da5b963c0e1e3d68a38...,8b39f00bfc9d45ed6f64dd39a72548936a157f9f03e7f4...,314d20e9e9ec3e97d1867c8bf8c6feb0c23d918021e175...,06a7f8e972f61aeb0e06335699518079a444e4450ff766...,c9f34437ce0e536fefd11a34b9a411b541d2dabfec872a...,bf2241c08d92d32a6782b4041a2c11ca58882ca88454b3...,702e4598004745673c0f6b50387bef9e1d5f503bd8c1c0...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,ea2f413bd8fda0b91a814a68aa520044b204796991a343...,...,0.000021,3.408966e-09,-0.001698,147.326389,148.659722,209.25,4,4,1,0


In [54]:
val.head()

,query_id,user_id,session_id,product_id,page_type,previous_page_type,device_category,device_platform,user_tier,user_country,...,diff_start_online_date_from_user_tier_clicked_mean,diff_prod_price_from_session_mean,diff_prod_price_from_query_mean,diff_prod_price_from_click_mean,diff_start_online_date_from_session_mean,diff_start_online_date_from_query_mean,diff_start_online_date_from_click_mean,price_rank,start_online_date_rank,user_step
0,45dbde6284f13d59b04c5d2a5ab2513c896ba8f7a7dde0...,4d66a7c430e1f1f7da454f4e8c4bf3e7cf2435741329c0...,b677570c68f211d9543e96fe46750b66ebdfaa0fb2df0e...,d52833e4925f40de987bb732847a8dbc07c2ba1e33711a...,06a7f8e972f61aeb0e06335699518079a444e4450ff766...,c9f34437ce0e536fefd11a34b9a411b541d2dabfec872a...,bf2241c08d92d32a6782b4041a2c11ca58882ca88454b3...,702e4598004745673c0f6b50387bef9e1d5f503bd8c1c0...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,95d5a1bd42a07143383299c081524278a16ef5ce55507c...,...,-150.511957,-0.000770,-1.390766e-03,-0.001335,-163.994444,-265.854167,-138.75,1,1,1
1,6728f8b02604af603d65faa0a2a4e36307f2498c4bdd88...,9ff86157cc29dd17330d2e714318770884adf936794c66...,d1f1a8e13d81de690a968db95dd1960482143f59e6c69b...,ca58ed0a66cb8990221552d0d93de82713641b66465a86...,06a7f8e972f61aeb0e06335699518079a444e4450ff766...,c9f34437ce0e536fefd11a34b9a411b541d2dabfec872a...,bf2241c08d92d32a6782b4041a2c11ca58882ca88454b3...,702e4598004745673c0f6b50387bef9e1d5f503bd8c1c0...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,c45f0825291fd1a087ef31abbcf9fd0ef10c915edf3041...,...,-12.511957,0.311558,-2.637422e-01,0.612305,-33.833333,-33.833333,-0.75,5,2,1
2,9d1d782fab18c98c8a8d4dab9cbc0cb3f786b5d5e5fd24...,b67c2b47e18777596747b94d0ae3e4a9f023a406fa5b5e...,d572b385d62d7eaaa4872e07ed2771b107d4db0b547212...,6374ef1b29e3046f9ae1607cd62cde1eb6305d4d9b1de5...,06a7f8e972f61aeb0e06335699518079a444e4450ff766...,14fc80e2d6821260d291ff47ce6d8d7534d4cb5aa4ab0d...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,58f6d97120664752ed0851301aa78457fe882f67453c58...,...,-258.511957,0.000271,-2.313843e-07,-0.000589,-223.534722,-223.534722,-246.75,6,1,1
3,f23d6751b37c235047a64a20ffe732483f487743dc8812...,8e28240e0b0c5629959d76727906afa17ea5d89821cbac...,c5df0c490099d1e4a20173ecbb67880998bc4faf8210a9...,e74667d17676e39561716943bc7cf8ba8a94ce96a0dc41...,06a7f8e972f61aeb0e06335699518079a444e4450ff766...,c9f34437ce0e536fefd11a34b9a411b541d2dabfec872a...,bf2241c08d92d32a6782b4041a2c11ca58882ca88454b3...,702e4598004745673c0f6b50387bef9e1d5f503bd8c1c0...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,f319f71fd9b14532cf715e601fe3b5c2473b58d4302e88...,...,-259.511957,0.000029,-6.575394e-06,-0.001369,-25.699074,-32.027778,-247.75,5,2,1
4,fe162740ab90f69597b14253e6f0fc6fb87ad8e2146cd8...,2173a7fa04d32abcb18cf8652e427f13a21156f7ff0bf5...,1e5f3aaf57b4863421d5019fe834f5e6979876f3d44e43...,a85298cf45012b8bfe8e5ec39108361e6083cad362dff3...,06a7f8e972f61aeb0e06335699518079a444e4450ff766...,c9f34437ce0e536fefd11a34b9a411b541d2dabfec872a...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,ea2f413bd8fda0b91a814a68aa520044b204796991a343...,...,-221.511957,0.000012,-4.198443e-05,-0.001639,-153.699074,-177.868056,-209.75,5,2,1


In [55]:
train.shape, val.shape

((3507990, 111), (687192, 110))